## Dependencies

In [1]:
import pymysql
pymysql.install_as_MySQLdb()

In [2]:
import pandas as pd
from sqlalchemy import create_engine
import json
from pprint import pprint
import requests

## Import Files

In [24]:
#Zipcode Lat/Lon
zipcodes_file = "zipcodes.csv"
zips_df = pd.read_csv(zipcodes_file)
zips_df.head()

#State Lat/Lon
states_file = "states.csv"
states_df = pd.read_csv(states_file)
states_df.head()

# Metric Data
## Anemia
anem_file = "Anemia Management.csv"
anem_df_1 = pd.read_csv(anem_file)
anem_df_1.head()

# Depression
depr_file = "Depression Screening.csv"
depr_df_1 = pd.read_csv(depr_file)
depr_df_1.head()

,Facility Name,CMS Certification Number (CCN),Alternate CCN 1,Address 1,Address 2,City,State,Zipcode,Network,Measure Name,Clinical Depression Screening and Follow-up Measure Score,State Avg Clinical Depression Screening and Follow-up Measure Score,National Avg Clinical Depression Screening and Follow-up Measure Score
0,CHILDRENS HOSPITAL DIALYSIS,12306,13300,1600 7TH AVENUE SOUTH,-,BIRMINGHAM,AL,35233,8,CLINICAL DEPRESSION,10,10,10
1,FMC CAPITOL CITY,12500,-,255 S JACKSON STREET,-,MONTGOMERY,AL,36104,8,CLINICAL DEPRESSION,10,10,10
2,GADSDEN DIALYSIS,12501,-,409 SOUTH FIRST STREET,-,GADSDEN,AL,35901,8,CLINICAL DEPRESSION,10,10,10
3,TUSCALOOSA UNIVERSITY DIALYSIS,12502,-,220 15TH STREET,-,TUSCALOOSA,AL,35401,8,CLINICAL DEPRESSION,10,10,10
4,PCD MONTGOMERY,12505,-,1001 FOREST AVENUE,-,MONTGOMERY,AL,36106,8,CLINICAL DEPRESSION,10,10,10


## Transform Data

In [41]:
# ANEMIA

# Merge Metric & Zipcode Lat/Lon Data
anem_merged_df = pd.merge(anem_df_1, zips_df, how = "left", on="Zipcode")
anem_merged_df = anem_merged_df[["Facility Name", "CMS Certification Number (CCN)", "City_x", "State_x", "Zipcode", "Measure Name", "Anemia Management Measure Score", "State Avg Anemia Management Measure Score", "National Avg Anemia Management Measure Score", "Lat", "Long"]]
anem_merged_df = anem_merged_df.rename(columns = {"Lat": "ZipLat", "Long" : "ZipLon", "State_x" : "state", "City_x" : "City"})
anem_merged_df.head()

# Merge in State Lat/Lon data
anem_df = pd.merge(anem_merged_df, states_df, left_on = "state", right_on = "State")
anem_df = anem_df.drop(columns=["City_y", "State"])
anem_df = anem_df[anem_df["Anemia Management Measure Score"] != "No Score"]
anem_df = anem_df.rename(columns = {"Facility Name" : "FacilityName", "CMS Certification Number (CCN)" : "CCN", "City_x" : "City", "Measure Name" : "MeasureName", "Anemia Management Measure Score" : "AnemiaScore", "State Avg Anemia Management Measure Score" : "value", "National Avg Anemia Management Measure Score" : "NationalAvg",  "Latitude" : "StateLat", "Longitude" : "StateLon"})
anem_df["AnemiaScore"] = anem_df["AnemiaScore"].astype('int64')
anem_df.head()

# State avgs 
anem_stateavgs_df = anem_df.groupby(["state"]).mean()
anem_stateavgs_df = anem_stateavgs_df[["value", "NationalAvg", "StateLat", "StateLon"]]
anem_stateavgs_df = anem_stateavgs_df.dropna()
anem_stateavgs_df

# Non-10 data
anem_outliers_df = anem_df[anem_df["AnemiaScore"] < 10]
anem_outliers_df.head()
anem_outliers_stateavgs_df = anem_outliers_df.groupby(["state"]).mean()
anem_outliers_stateavgs_df = anem_outliers_stateavgs_df[["AnemiaScore", "StateLat", "StateLon"]]
anem_outliers_stateavgs_df = anem_outliers_stateavgs_df.rename(columns = {"AnemiaScore" : "value"})
anem_outliers_stateavgs_df = anem_outliers_stateavgs_df.round(2)
anem_outliers_stateavgs_df

#------------------------------------------
# DEPRESSION

# Merge Metric & Zipcode Lat/Lon Data
depr_merged_df = pd.merge(depr_df_1, zips_df, how = "left", on="Zipcode")
depr_merged_df = depr_merged_df[["Facility Name", "CMS Certification Number (CCN)", "City_x", "State_x", "Zipcode", "Measure Name", "Clinical Depression Screening and Follow-up Measure Score", "State Avg Clinical Depression Screening and Follow-up Measure Score", "National Avg Clinical Depression Screening and Follow-up Measure Score", "Lat", "Long"]]
depr_merged_df = depr_merged_df.rename(columns = {"Lat": "ZipLat", "Long" : "ZipLon", "State_x" : "state", "City_x" : "City"})
depr_merged_df.head()

# Merge in State Lat/Lon data
depr_df = pd.merge(depr_merged_df, states_df, left_on = "state", right_on = "State")
depr_df = depr_df.drop(columns=["City_y", "State"])
depr_df = depr_df[depr_df["Clinical Depression Screening and Follow-up Measure Score"] != "No Score"]
depr_df = depr_df.rename(columns = {"Facility Name" : "FacilityName", "CMS Certification Number (CCN)" : "CCN", "City_x" : "City", "Measure Name" : "MeasureName", "Clinical Depression Screening and Follow-up Measure Score" : "DepressionScore", "State Avg Clinical Depression Screening and Follow-up Measure Score" : "value", "National Avg Clinical Depression Screening and Follow-up Measure Score" : "NationalAvg",  "Latitude" : "StateLat", "Longitude" : "StateLon"})
depr_df.head()

# State avgs 
depr_stateavgs_df = depr_df.groupby(["state"]).mean()
depr_stateavgs_df = depr_stateavgs_df[["value", "NationalAvg", "StateLat", "StateLon"]]
depr_stateavgs_df = depr_stateavgs_df.dropna()
depr_stateavgs_df

anem_df.head()
# anem_df.dtypes
anem_outliers_stateavgs_df

,value,StateLat,StateLon
state,,,
AL,8.00,32.60,-86.68
AR,8.00,34.75,-92.13
AZ,9.00,34.17,-111.93
CA,7.00,37.27,-119.27
CO,9.00,39.00,-105.55
CT,9.00,41.52,-72.76
DC,9.00,38.90,-77.01
FL,8.11,27.98,-83.83
GA,8.86,32.68,-83.22


## Database

In [26]:
connection_string = "root:toor2@localhost/esrd_db"
engine = create_engine(f'mysql://{connection_string}')

In [27]:
# Confirm tables
engine.table_names()

['anem_stateavgs_tbl',
 'anem_stateoutliers_tbl',
 'anem_tbl',
 'depr_stateavgs_tbl',
 'depr_tbl']

In [28]:
anem_df.to_sql(name='anem_tbl', con=engine, if_exists='append', index=False)

In [29]:
anem_stateavgs_df.to_sql(name='anem_stateavgs_tbl', con=engine, if_exists='append', index=False)

In [33]:
anem_outliers_stateavgs_df.to_sql(name='anem_stateoutliers_tbl', con=engine, if_exists='append', index=False)

In [34]:
depr_df.to_sql(name='depr_tbl', con=engine, if_exists='append', index=False)

In [35]:
depr_stateavgs_df.to_sql(name='depr_stateavgs_tbl', con=engine, if_exists='append', index=False)

In [36]:
anem_stateavgs_df.to_csv("anem_stateavgs.csv")

In [42]:
anem_outliers_stateavgs_df.to_csv("anem_stateoutliers.csv")